**Import Library**

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import sympy as sp
from PIL import Image, ImageOps
import os

**Function**

In [90]:
#Process the Image into a flat 1D array
def ProcessImage(img):
    Grascale = ImageOps.grayscale(img)
    Facematrix = im.pil_to_array(Grascale)
    FlatFaceArray = np.concat(Facematrix,axis=None)
    return FlatFaceArray

#Make the Data Matrix

def Datamatrix(clsdir: str, num_files: int):
    for i in range(num_files):
        imgdir = clsdir +"\\" + str(i+1) + ".pgm"
    #Nhap hinh anh
        Face = Image.open(imgdir)
    #Process hinh anh
        FlatFaceArray = ProcessImage(Face)
        if i == 0:
            FaceMatrix = FlatFaceArray
        else:
            FaceMatrix = np.column_stack((FaceMatrix,FlatFaceArray))
    return FaceMatrix

#Make a Data Matrix for a class k
def clsMatrix(k: int):
    clsdir = "archive\\s" + str(k)
    num_files = sum(1 for entry in os.scandir(clsdir) if entry.is_file())
    return Datamatrix(clsdir, num_files)
clsMatrix(2).shape

#number of class
num_class = sum(1 for entry in os.scandir("archive") if not entry.is_file())

for i in range(num_class):
    if i == 0:
        FullMatrix = clsMatrix(i+1)
    else:
        Fullmatrix = np.append(FullMatrix, clsMatrix(i+1),axis=1)
        a = i
num_class
a
Fullmatrix.shape

(10304, 20)

**Size of the data**

In [42]:
imgdir = "archive\\s1\\1.pgm"
Face= Image.open(imgdir)
FaceArray = im.pil_to_array(Face)
Height, Width = FaceArray.shape


**Make the overall Data Matrix**

In [116]:
#number of class
num_class = sum(1 for entry in os.scandir("archive") if not entry.is_file())

for i in range(11):
    if i == 0:
        FullMatrix = clsMatrix(i+1)
    else:
        Fullmatrix = np.append(clsMatrix(i), clsMatrix(i+1),axis=1)
FullMatrix.shape

(10304, 10)

In [ ]:
#The average face
MeanFace=np.mean(FaceMatrix, axis=1)
ColumnMeanFace= MeanFace[:, np.newaxis]
print(MeanFace)

[47.8 49.7 47.2 ... 49.9 46.9 49. ]


(10304,)

In [79]:
CenteredFaceMatrix=FaceMatrix  - ColumnMeanFace
#Find the Eigenface of the Data
BtB=CenteredFaceMatrix.T @ CenteredFaceMatrix
V, S, Vt = np.linalg.svd(BtB)
V
S_matrix=np.sqrt(np.diag(S))
S_inv=np.linalg.inv(S_matrix)
U=CenteredFaceMatrix @ V @ S_inv
CenteredFaceMatrix.shape

(10304, 10)

In [ ]:
#Reconstruct the Eigenface as a picture
Eigenface=U @ S_matrix +ColumnMeanFace
face = Eigenface[:,1]
face = np.split(face, Height)
for i in range(len(face)):
    if i == 0:
        recface=face[0]
    else:
        recface = np.vstack((recface,face[i]))
recim = Image.fromarray(recface).convert("L")
recim.save('test.png','PNG')

In [507]:
#let G be the face that need to be recognize
G = np.random.rand(U[:,1].shape[0])
# the cotribution of a eigenface to it is calculated by
#Let Omega be the contribution vector

def Facespace(vector):
    Omega = np.array([])
    for i in range(num_files):
        diff=U[:,i] @ vector
        Omega = np.append(Omega,[[diff]] )
    return Omega
#Omega= Omega[:, np.newaxis]
#Find the threshhold for the picture to be identified as a face, let that be epsilon
epsilon=0
S = 0
for i in range(num_files):
    S+= Facespace(CenteredFaceMatrix[:,i])
    avgOmega = S/len(range(num_files))
for i in range(num_files):
    e = np.linalg.norm(Facespace(CenteredFaceMatrix[:,i])-avgOmega)
    if e > epsilon:
        epsilon = e

In [508]:

#Input = ProcessImage(Image.open("32.JPG"))-MeanFace
#Input.shape

image=Image.open("32.JPG")
Omega = Facespace(ProcessImage(image)-MeanFace)
epsilon > np.linalg.norm(Omega-avgOmega)


np.False_